In [2]:
import numpy as np
import pandas as pd
import os.path
from random import randint

ratings_file = './data/ratings.csv'
predictions_file = './data/predictions.csv'

ratings_description = pd.read_csv(ratings_file, delimiter=';',
                                  dtype={'userID': 'int', 'movieID': 'int', 'rating': 'int'},
                                  names=['userID', 'movieID', 'rating'])

num_movies = max(ratings_description["movieID"])
num_users = max(ratings_description["userID"])
R = np.zeros(( num_movies, num_users))

for user, movie, rating in ratings_description.values:
    R[movie-1, user-1] = rating

R[R==0] = np.nan
print(f"Shape of Utility matrix is (movies, users): {R.shape}")
R

Shape of Utility matrix is (movies, users): (3706, 6040)


array([[ 5., nan, nan, ..., nan, nan,  3.],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       ...,
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan],
       [nan, nan, nan, ..., nan, nan, nan]])

In [8]:
#### LATENT FACTORS HYPERPARAMETERS ####
from tqdm import tqdm_notebook as tqdm

EPOCHS = 1500
LEARNING_RATE = 0.05 # == nu
K = 2 # number of factors to work with.

np.random.seed(42)
Q = np.random.uniform(-1, 1, (R.shape[0], K))
P = np.random.uniform(-1, 1, (K, R.shape[1]))
div = (R.shape[0] * R.shape[1]) - np.isnan(R).sum()
RMSE = np.sqrt(((np.nan_to_num(R - np.matmul(Q, P), 0)**2).sum())/div)
print(f"Starting RMSE: {RMSE}")

lf_learning_curve = []

for epoch in tqdm(range(1500)):
    R_pred = np.matmul(Q,P)
    curr_error = np.nan_to_num(R - R_pred, 0)
    Q_update = np.zeros(Q.shape)
    for i in range(len(Q_update)):
        for curr_k in range(K):
            Q_delta =(-2 * np.dot(P[curr_k, :], curr_error[i]))/np.isnan(R[i]).sum()
            Q_update[i, curr_k] = LEARNING_RATE * Q_delta

    P_update = np.zeros(P.shape)
    for i in range(P_update.shape[1]):
        for curr_k in range(K):
            P_delta =(-2 * np.dot(Q[:, curr_k], curr_error[:, i]))/np.isnan(R[:, i]).sum()
            P_update[curr_k, i] = LEARNING_RATE * P_delta

    Q -= Q_update
    P -= P_update
    
    RMSE_i = np.sqrt(((np.nan_to_num(R - np.matmul(Q, P), 0)**2).sum())/div)
    print(f"RMSE {epoch}: {RMSE_i}")
    lf_learning_curve.append([epoch, RMSE_i])


RMSE = np.sqrt(((np.nan_to_num(R - np.matmul(Q, P), 0)**2).sum())/div)
print(f"Final RMSE: {RMSE}")

Starting RMSE: 3.781830053581588


<ipython-input-8-b06c0ec8f32b>:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for epoch in tqdm(range(1500)):


RMSE 0: 3.781116582770836
RMSE 1: 3.7804303543640434
RMSE 2: 3.7797696986106315
RMSE 3: 3.779133075766699
RMSE 4: 3.7785190638260566
RMSE 5: 3.777926347518714
RMSE 6: 3.7773537084319457
RMSE 7: 3.776800016126702
RMSE 8: 3.7762642201374477
RMSE 9: 3.775745342756671
RMSE 10: 3.7752424725173173
RMSE 11: 3.7747547582959062
RMSE 12: 3.774281403968534
RMSE 13: 3.773821663559242
RMSE 14: 3.7733748368272275
RMSE 15: 3.77294026524525
RMSE 16: 3.7725173283266837
RMSE 17: 3.772105440263456
RMSE 18: 3.7717040468407665
RMSE 19: 3.771312622598418
RMSE 20: 3.7709306682113155
RMSE 21: 3.770557708064592
RMSE 22: 3.7701932880012943
RMSE 23: 3.769836973222294
RMSE 24: 3.7694883463204754
RMSE 25: 3.769147005432286
RMSE 26: 3.7688125624915463
RMSE 27: 3.7684846415715736
RMSE 28: 3.768162877302465
RMSE 29: 3.7678469133517347
RMSE 30: 3.7675364009568826
RMSE 31: 3.767230997499459
RMSE 32: 3.7669303651104733
RMSE 33: 3.7666341692977707
RMSE 34: 3.766342077585965
RMSE 35: 3.7660537581602505
RMSE 36: 3.76576887

RMSE 291: 1.3331636290709301
RMSE 292: 1.3297624196940512
RMSE 293: 1.3264001580128055
RMSE 294: 1.3230762486560903
RMSE 295: 1.3197901077052796
RMSE 296: 1.3165411624326595
RMSE 297: 1.3133288510465848
RMSE 298: 1.3101526224431266
RMSE 299: 1.307011935964068
RMSE 300: 1.3039062611610626
RMSE 301: 1.3008350775657849
RMSE 302: 1.2977978744659526
RMSE 303: 1.2947941506869658
RMSE 304: 1.2918234143791452
RMSE 305: 1.288885182810293
RMSE 306: 1.285978982163516
RMSE 307: 1.283104347340119
RMSE 308: 1.2802608217674696
RMSE 309: 1.2774479572116726
RMSE 310: 1.2746653135949104
RMSE 311: 1.271912458817389
RMSE 312: 1.2691889685836888
RMSE 313: 1.2664944262334337
RMSE 314: 1.263828422576178
RMSE 315: 1.2611905557303658
RMSE 316: 1.2585804309662667
RMSE 317: 1.2559976605527963
RMSE 318: 1.253441863608086
RMSE 319: 1.2509126659537184
RMSE 320: 1.248409699972543
RMSE 321: 1.2459326044699364
RMSE 322: 1.2434810245384635
RMSE 323: 1.2410546114258076
RMSE 324: 1.2386530224059105
RMSE 325: 1.2362759206

RMSE 576: 0.99164871653872
RMSE 577: 0.9912923669010332
RMSE 578: 0.9909380648568363
RMSE 579: 0.9905857952176995
RMSE 580: 0.990235542932122
RMSE 581: 0.9898872930840631
RMSE 582: 0.989541030891514
RMSE 583: 0.9891967417050577
RMSE 584: 0.9888544110064778
RMSE 585: 0.9885140244073498
RMSE 586: 0.988175567647689
RMSE 587: 0.9878390265945828
RMSE 588: 0.9875043872408547
RMSE 589: 0.9871716357037468
RMSE 590: 0.986840758223599
RMSE 591: 0.9865117411625812
RMSE 592: 0.9861845710033963
RMSE 593: 0.985859234348037
RMSE 594: 0.9855357179165355
RMSE 595: 0.9852140085457342
RMSE 596: 0.984894093188076
RMSE 597: 0.9845759589104035
RMSE 598: 0.9842595928927667
RMSE 599: 0.9839449824272652
RMSE 600: 0.9836321149168898
RMSE 601: 0.9833209778743667
RMSE 602: 0.9830115589210462
RMSE 603: 0.9827038457857747
RMSE 604: 0.9823978263038016
RMSE 605: 0.9820934884156822
RMSE 606: 0.9817908201662071
RMSE 607: 0.9814898097033372
RMSE 608: 0.9811904452771579
RMSE 609: 0.9808927152388321
RMSE 610: 0.9805966080

RMSE 860: 0.937973000389006
RMSE 861: 0.9378785406877633
RMSE 862: 0.9377844387679213
RMSE 863: 0.9376906928011107
RMSE 864: 0.9375973009702893
RMSE 865: 0.9375042614696627
RMSE 866: 0.9374115725046005
RMSE 867: 0.9373192322915556
RMSE 868: 0.9372272390579905
RMSE 869: 0.9371355910422813
RMSE 870: 0.9370442864936565
RMSE 871: 0.9369533236721067
RMSE 872: 0.9368627008483139
RMSE 873: 0.9367724163035639
RMSE 874: 0.9366824683296845
RMSE 875: 0.936592855228961
RMSE 876: 0.9365035753140557
RMSE 877: 0.9364146269079463
RMSE 878: 0.9363260083438384
RMSE 879: 0.9362377179651031
RMSE 880: 0.9361497541251992
RMSE 881: 0.9360621151875977
RMSE 882: 0.9359747995257175
RMSE 883: 0.9358878055228502
RMSE 884: 0.935801131572085
RMSE 885: 0.9357147760762495
RMSE 886: 0.9356287374478293
RMSE 887: 0.935543014108914
RMSE 888: 0.935457604491106
RMSE 889: 0.9353725070354753
RMSE 890: 0.9352877201924807
RMSE 891: 0.9352032424219074
RMSE 892: 0.9351190721927953
RMSE 893: 0.9350352079833847
RMSE 894: 0.9349516

RMSE 1139: 0.9209365650262554
RMSE 1140: 0.9208977127532795
RMSE 1141: 0.9208589630189786
RMSE 1142: 0.9208203154192482
RMSE 1143: 0.9207817695518989
RMSE 1144: 0.9207433250166147
RMSE 1145: 0.9207049814149711
RMSE 1146: 0.9206667383504019
RMSE 1147: 0.9206285954282184
RMSE 1148: 0.9205905522555604
RMSE 1149: 0.9205526084414177
RMSE 1150: 0.9205147635966007
RMSE 1151: 0.9204770173337444
RMSE 1152: 0.9204393692672866
RMSE 1153: 0.9204018190134643
RMSE 1154: 0.9203643661903046
RMSE 1155: 0.9203270104176045
RMSE 1156: 0.9202897513169367
RMSE 1157: 0.9202525885116254
RMSE 1158: 0.9202155216267487
RMSE 1159: 0.9201785502891197
RMSE 1160: 0.9201416741272789
RMSE 1161: 0.9201048927714917
RMSE 1162: 0.9200682058537261
RMSE 1163: 0.920031613007652
RMSE 1164: 0.9199951138686332
RMSE 1165: 0.9199587080737167
RMSE 1166: 0.919922395261616
RMSE 1167: 0.9198861750727091
RMSE 1168: 0.9198500471490292
RMSE 1169: 0.9198140111342553
RMSE 1170: 0.9197780666737023
RMSE 1171: 0.9197422134143058
RMSE 1172: 0

RMSE 1414: 0.9130687675299558
RMSE 1415: 0.9130475708880104
RMSE 1416: 0.9130264121079142
RMSE 1417: 0.913005291061696
RMSE 1418: 0.9129842076218447
RMSE 1419: 0.9129631616613204
RMSE 1420: 0.9129421530535554
RMSE 1421: 0.9129211816724431
RMSE 1422: 0.912900247392341
RMSE 1423: 0.9128793500880682
RMSE 1424: 0.9128584896349018
RMSE 1425: 0.9128376659085743
RMSE 1426: 0.9128168787852727
RMSE 1427: 0.9127961281416401
RMSE 1428: 0.9127754138547673
RMSE 1429: 0.9127547358021935
RMSE 1430: 0.9127340938619062
RMSE 1431: 0.9127134879123338
RMSE 1432: 0.912692917832352
RMSE 1433: 0.9126723835012704
RMSE 1434: 0.9126518847988452
RMSE 1435: 0.9126314216052612
RMSE 1436: 0.9126109938011442
RMSE 1437: 0.912590601267547
RMSE 1438: 0.9125702438859558
RMSE 1439: 0.9125499215382878
RMSE 1440: 0.91252963410688
RMSE 1441: 0.9125093814745046
RMSE 1442: 0.9124891635243462
RMSE 1443: 0.9124689801400144
RMSE 1444: 0.9124488312055402
RMSE 1445: 0.9124287166053705
RMSE 1446: 0.9124086362243675
RMSE 1447: 0.912